In [4]:
import numpy as np
import pandas_datareader as pdr
import datetime
import pandas as pd
from datetime import datetime

In [19]:
def get(tickers, startdate, enddate):
    def data(ticker):
        return pdr.get_data_google(ticker, start=startdate, end=enddate)

    datas = map(data, tickers)
    return pd.concat(datas, keys=tickers, names=['Ticker', 'Date'])

all_data = get(['AAPL', 'IBM', 'GOOG'], datetime(2007, 1, 1), datetime(2017, 1, 1))

In [41]:
all_data.head(2)

Open   High    Low  Close       Volume
Ticker Date                                               
AAPL   2007-01-03  12.33  12.37  11.70  11.97  311433248.0
       2007-01-04  12.01  12.28  11.97  12.24  214031636.0

In [23]:
all_data_flat = all_data.reset_index()

In [40]:
all_data_flat.head(2)

,Ticker,Date,Open,High,Low,Close,Volume
0,AAPL,2007-01-03,12.33,12.37,11.70,11.97,311433248.0
1,AAPL,2007-01-04,12.01,12.28,11.97,12.24,214031636.0


In [39]:
all_data_flat.pivot(index='Date', columns='Ticker', values='Close').head(2)

Ticker,AAPL,GOOG,IBM
Date,,,
2007-01-03,11.97,233.56,97.27
2007-01-04,12.24,241.39,98.31


In [44]:
all_data_unstacked = all_data.unstack(level=0)
all_data_unstacked.head(2)

Open                  High                   Low                 \
Ticker       AAPL    IBM    GOOG   AAPL    IBM    GOOG   AAPL    IBM    GOOG   
Date                                                                           
2007-01-03  12.33  97.18  232.77  12.37  98.40  238.09  11.70  96.26  230.32   
2007-01-04  12.01  97.25  234.27  12.28  98.79  241.73  11.97  96.88  233.94   

            Close                      Volume                   
Ticker       AAPL    IBM    GOOG         AAPL         IBM GOOG  
Date                                                            
2007-01-03  11.97  97.27  233.56  311433248.0   9199500.0  NaN  
2007-01-04  12.24  98.31  241.39  214031636.0  10557200.0  NaN

In [54]:
all_data_unstacked['Close', 'AAPL'].head(2)

Date
2007-01-03    11.97
2007-01-04    12.24
Name: (Close, AAPL), dtype: float64

In [63]:
all_data_unstacked_pct = all_data_unstacked.pct_change().fillna(0)
all_data_unstacked_pct.head(2)

Open                         High                      \
Ticker          AAPL      IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                    
2007-01-03  0.000000  0.00000  0.000000  0.000000  0.000000  0.000000   
2007-01-04 -0.025953  0.00072  0.006444 -0.007276  0.003963  0.015288   

                 Low                         Close                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007-01-04  0.023077  0.006441  0.015717  0.022556  0.010692  0.033525   

              Volume                 
Ticker          AAPL       IBM GOOG  
Date                                 
2007-01-03  0.000000  0.000000  0.0  
2007-01-04 -0.312753  0.147584  0.0

In [65]:
from sklearn.model_selection import train_test_split

In [88]:
rows = all_data_unstacked_pct.shape[0]
train_size = int(rows*0.75)
test_size = rows-train_size
print('Train: %d, test: %d' % (train_size, test_size))

Train: 1888, test: 630


In [89]:
train_X = all_data_unstacked_pct.iloc[0:train_size]
test_X = all_data_unstacked_pct.iloc[train_size:]
assert train_X.shape[0] + test_X.shape[0] == rows

In [90]:
train_X.head()

Open                          High                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007-01-04 -0.025953  0.000720  0.006444 -0.007276  0.003963  0.015288   
2007-01-05  0.019983  0.003599  0.028770  0.002443 -0.008503  0.007364   
2007-01-08  0.002449  0.009221  0.010746  0.004062  0.015824  0.004846   
2007-01-09  0.005700  0.005888 -0.004598  0.074434  0.008342 -0.003310   

                 Low                         Close                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007-01-04  0.023077  0.006441  0.015717  0.022556  0.010692  0.033525   
2007-01-05  0.007519  0.000310  0.020860 -0.007353 -0.009053  0.008120   
2007-01-08  0.009950  0.014859  0.008542  0.004938  0.015192 -0.007397   
2007-01-09 -0.001642  0.007321 -0.002076  0.082719  0.011830  0.003974   

              Volume                 
Ticker          AAPL       IBM GOOG  
Date                                 
2007-01-03  0.000000  0.000000  0.0  
2007-01-04 -0.312753  0.147584  0.0  
2007-01-05 -0.024363 -0.315832  0.0  
2007-01-08 -0.044947  0.431572  0.0  
2007-01-09  3.202131  0.074351  0.0

In [91]:
from sklearn.preprocessing import StandardScaler

In [93]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [101]:
train_X_scaled = pd.DataFrame(data=train_X_scaled, columns=train_X.columns)
test_X_scaled = pd.DataFrame(data=test_X_scaled, columns=test_X.columns)
train_X_scaled.head()

Open                          High                           Low  \
Ticker      AAPL       IBM      GOOG      AAPL       IBM      GOOG      AAPL   
0      -0.056854 -0.031304 -0.028985 -0.067191 -0.034376 -0.036396 -0.059790   
1      -1.141226  0.019096  0.169111 -0.459282  0.289591  0.860756  0.955350   
2       0.778095  0.220515  0.855429  0.064464 -0.729396  0.395715  0.270957   
3       0.045470  0.613910  0.301367  0.151699  1.259100  0.247966  0.377915   
4       0.181319  0.380701 -0.170321  3.944098  0.647470 -0.230652 -0.132023   

                               Close                        Volume            \
Ticker       IBM      GOOG      AAPL       IBM      GOOG      AAPL       IBM   
0      -0.032351 -0.035378 -0.060116 -0.031200 -0.034167 -0.136343 -0.146259   
1       0.436986  0.800880  0.953554  0.697625  1.648268 -0.985940  0.238283   
2      -0.009787  1.074509 -0.390553 -0.648307  0.373318 -0.202526 -0.969182   
3       1.050411  0.419110  0.161807  1.004377 -0.405374 -0.258443  0.978234   
4       0.501103 -0.145829  3.657230  0.775215  0.165285  8.562293  0.047469   

                  
Ticker      GOOG  
0      -0.021249  
1      -0.021249  
2      -0.021249  
3      -0.021249  
4      -0.021249

In [107]:
train_X_scaled.describe()

Open                                      High                \
Ticker          AAPL           IBM          GOOG          AAPL           IBM   
count   1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03   
mean    6.615471e-18  4.586726e-18  1.709732e-17  1.308393e-18  1.511268e-17   
std     1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00   
min    -6.113093e+00 -6.917997e+00 -1.525398e+01 -6.788799e+00 -5.891316e+00   
25%    -4.933883e-01 -4.569576e-01 -2.836463e-01 -4.923048e-01 -4.557095e-01   
50%     1.381718e-02  1.822354e-02 -8.202510e-03 -6.229087e-03 -3.007348e-05   
75%     5.054681e-01  5.256086e-01  2.860741e-01  4.807749e-01  4.355084e-01   
max     9.327024e+00  5.418440e+00  3.125474e+01  6.747618e+00  6.395425e+00   

                               Low                                     Close  \
Ticker          GOOG          AAPL           IBM          GOOG          AAPL   
count   1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03   
mean   -4.777840e-18 -3.733331e-17 -4.116293e-17  1.017312e-17  3.969283e-19   
std     1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00   
min    -5.316449e+00 -8.813099e+00 -6.424892e+00 -5.130838e+00 -8.106035e+00   
25%    -4.393732e-01 -4.507512e-01 -4.166981e-01 -4.511574e-01 -4.878459e-01   
50%    -2.203250e-02  3.089916e-02  1.859382e-02  2.704752e-02  4.863213e-04   
75%     4.511142e-01  4.517891e-01  4.595603e-01  4.667880e-01  5.236372e-01   
max     1.124686e+01  8.410895e+00  6.179507e+00  9.287179e+00  6.290016e+00   

                                          Volume                              
Ticker           IBM          GOOG          AAPL           IBM          GOOG  
count   1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  
mean    2.528580e-18 -1.411300e-18 -2.469776e-18 -2.128712e-17  6.381430e-16  
std     1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  
min    -5.674692e+00 -5.859183e+00 -2.052510e+00 -2.291763e+00 -9.532627e+00  
25%    -4.717483e-01 -4.416886e-01 -6.343269e-01 -6.055594e-01 -2.124889e-02  
50%    -8.295980e-03 -2.090848e-02 -1.772898e-01 -1.761633e-01 -2.124889e-02  
75%     5.094054e-01  4.582160e-01  4.148680e-01  3.724554e-01 -2.124889e-02  
max     7.818135e+00  9.998386e+00  1.369210e+01  1.253017e+01  2.070746e+01

In [109]:
train_X_scaled.max()

        Ticker
Open    AAPL       9.327024
        IBM        5.418440
        GOOG      31.254744
High    AAPL       6.747618
        IBM        6.395425
        GOOG      11.246862
Low     AAPL       8.410895
        IBM        6.179507
        GOOG       9.287179
Close   AAPL       6.290016
        IBM        7.818135
        GOOG       9.998386
Volume  AAPL      13.692099
        IBM       12.530170
        GOOG      20.707462
dtype: float64